# What is around each of top 100 world-class universities?

In [1]:
import pandas as pd
import numpy as np
import html5lib
import lxml
import re
import requests
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import matplotlib.pyplot as plt
import folium
import time
import geocoder
from tqdm import tqdm
import sys
import linecache

In [2]:
option = webdriver.ChromeOptions()
chrome_prefs = {}
option.experimental_options["prefs"] = chrome_prefs
option.add_argument('--headless')
option.add_argument('--no-sandbox')
option.add_argument('--disable-dev-shm-usage')
chrome_prefs["profile.default_content_settings"] = {"images": 2}
chrome_prefs["profile.managed_default_content_settings"] = {"images": 2}

In [3]:
url = 'https://www.topuniversities.com/university-rankings/world-university-rankings/2020'

In [4]:
tableCSSselector = '.quicktabs_main'
clickPerPageCSSselector = '#qs-rankings_length > label > span.jcf-select.jcf-unselectable > span.jcf-select-opener'
click100CSSselector = '#qs-rankings_length > label > span.jcf-select.jcf-unselectable.jcf-drop-active > div > div > span > span > ul > li:nth-child(3) > span'

In [5]:
def PrintException():
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    print('Exception in ({}, line {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj))

In [6]:
driver = webdriver.Chrome() #options = option)
driver.get(url)

wait = WebDriverWait(driver, 20)
wait.until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, tableCSSselector)))
print('ready ro scrape.')

driver.execute_script("window.scrollTo(0, 2100)")

time.sleep(1)

cursor = driver.find_element_by_css_selector(clickPerPageCSSselector)
cursor.click()
time.sleep(2)

cursor = driver.find_element_by_css_selector(click100CSSselector)
cursor.click()
time.sleep(5)

print('Top 100 institutions shown')

soup = BeautifulSoup(driver.page_source, 'html5lib')

QSurl = 'https://www.topuniversities.com'

uniName = []
uniLink = []
for i in soup.findAll('a', class_='title'):
    uniName.append(i.text)
    uniLink.append(i.get('href'))

mapCanvasXpath = '//*[@id="scene"]/div[3]/canvas'
whatsHereCSSselector = '#action-menu > ul > li:nth-child(3)'
LatLonCSSselector = '#reveal-card > div > div.widget-reveal-card-container > button.link-like.widget-reveal-card-lat-lng'

uniLocList = []
LatLonList = []

for i in tqdm(range(len(uniName))):
    try:
        urlUni = QSurl + uniLink[i]
        driver.get(urlUni)
    
        soupUni = BeautifulSoup(driver.page_source, 'html5lib')
    
        uniLocRaw = soupUni.select_one('.location')
        if uniLocRaw == None:
            locCSSselectorAltern = '#block-system-main > div > div.basic-pro > div.lead_section > header > div.container > div.title_info > div.location'
            uniLocRaw = soupUni.select_one(locCSSselectorAltern)

        uniLoc = re.findall('(?!\nView\smap).+[^View\smap]', uniLocRaw.text)
    
#     urlUniLoc = uniLoc.find('a').get('href')
#     gmapAPI = urlUniLoc[urlUniLoc.find('//')+2:]
    
#     driver.get('https://' + gmapAPI)
    
#     wait = WebDriverWait(driver, 30)
#     wait.until(EC.visibility_of_all_elements_located((By.XPATH, mapCanvasXpath)))
    
#     action = ActionChains(driver)
#     action.move_to_element(driver.find_element_by_xpath(mapCanvasXpath)).perform()
#     action.context_click().perform()
#     time.sleep(2)
    
#     cursor = driver.find_element_by_css_selector(whatsHereCSSselector)
#     cursor.click()
#     time.sleep(2)
    
#     soupGmaps = BeautifulSoup(driver.page_source, 'html5lib')

#     LatLon = soupGmaps.select_one(LatLonCSSselector).get('aria-label')
#     LatLon = list(map(float, LatLon.split(', ')))
    
        uniLocList.append(uniLoc)
#     LatLonList.append(LatLon)
        
        print(str(i+1) + ': ' + uniName[i])
        time.sleep(2)
    except:
        print('Error occured when scraping ' + uniName[i])
        PrintException()
        print('Continue to the next iteration.')
        uniLocList.append('')
        continue
    
driver.close()

ready ro scrape.
Top 100 institutions shown


  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

1: Massachusetts Institute of Technology (MIT)


  1%|▊                                                                                 | 1/100 [00:06<10:14,  6.20s/it]

2: Stanford University


...

 98%|███████████████████████████████████████████████████████████████████████████████▍ | 98/100 [23:17<00:28, 14.08s/it]

99: KTH Royal Institute of Technology


 99%|████████████████████████████████████████████████████████████████████████████████▏| 99/100 [23:29<00:13, 13.58s/it]

100: University of St Andrews


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [23:42<00:00, 14.23s/it]


In [26]:
uniLocList

[['77 Massachusetts Avenue Cambridge, United States'],
 ['450 Serra Mall Stanford, United States'],
 ['1350 Massachusetts Avenue Cambridge, United States'],
 ['Wellington Square Oxford, United Kingdo'],
 ['1200 East California Boulevard Pasadena, United States'],
 ['Raemistrasse 101 Zürich, Switzerland'],
 ['The Old Schools, Trinity Lane Cambridge, United Kingdo'],
 ['University College London, Gower Street London, United Kingdo'],
 ['South Kensington London, United Kingdo'],
 ['Edward H. Levi Hall, 5801 South Ellis Avenue Chicago, United States'],
 ['50 Nanyang Avenue Singapore, Singapor'],
 ['21 Lower Kent Ridge Road Singapore, Singapor'],
 ['Princeton University Princeton, United States'],
 ['144 East Avenue Ithaca, United States'],
 ['University of Pennsylvania Philadelphia, United States'],
 ['30 Shuangqing Rd Beijing, Chin'],
 ['Yale University New Haven, United States'],
 ['Columbia University 116th Street & Broadway New York, United States'],
 ['Ecole Polytechnique Federale de 

In [7]:
import geocoder
g = geocoder.arcgis('77 Massachusetts Avenue, Cambridge')
g.latlng

[42.35891737166005, -71.09346063887153]

In [5]:
g

<[ERROR - HTTPSConnectionPool(host='sgws2.maps.yahoo.com', port=443): Read timed out. (read timeout=5.0)] Yahoo - Geocode [empty]>